In [13]:
!pip -q install elevenlabs langchain-core requests

In [14]:
import os
import time
import requests
from typing import Optional
from langchain_core.runnables import RunnableLambda


In [22]:

os.environ["ELEVENLABS_API_KEY"] = os.getenv("ELEVENLABS_API_KEY", "")
#add the elevenlabs_api_here

In [23]:

FLORES_MAP = {
    "en": "eng_Latn",
    "hi": "hin_Deva",
    "bn": "ben_Beng",
    "ta": "tam_Taml",
    "te": "tel_Telu",
    "kn": "kan_Knda",
    "ml": "mal_Mlym",
    "mr": "mar_Deva",
    "gu": "guj_Gujr",
    "pa": "pan_Guru",
    "or": "ory_Orya",
    "as": "asm_Beng",
    "ne": "npi_Deva",
    "ur": "urd_Arab",
}

NLLB_BASE = "https://winstxnhdw-nllb-api.hf.space"
NLLB_TRANSLATE = f"{NLLB_BASE}/api/v4/translator"

def translate_text_nllb_api(english_text: str, target_lang_code: str, retries: int = 2, timeout: int = 60) -> str:
    """
    Translate English -> target_lang_code using HF Space NLLB API.
    target_lang_code like 'hi', 'ta', 'bn', etc. (mapped to FLORES codes).
    """
    target = FLORES_MAP.get(target_lang_code, target_lang_code)
    params = {
        "text": english_text,
        "source": FLORES_MAP["en"],
        "target": target
    }
    for attempt in range(retries + 1):
        try:
            r = requests.get(NLLB_TRANSLATE, params=params, timeout=timeout)
            r.raise_for_status()
            data = r.json()
            out = data.get("text") or data.get("result") or ""
            return out.strip()
        except Exception as e:
            if attempt < retries:
                time.sleep(1.5 * (attempt + 1))
                continue
            print(f"[ERROR] NLLB API translation failed: {e}")
            return ""

def translate_chunked_nllb(english_text: str, target_lang_code: str, max_chars: int = 900) -> str:
    text = english_text.strip()
    if len(text) <= max_chars:
        return translate_text_nllb_api(text, target_lang_code)

    parts = []
    buf = []
    cur = 0
    for sent in text.split(". "):
        add = (sent + ". ").strip()
        if cur + len(add) > max_chars and buf:
            parts.append(" ".join(buf).strip())
            buf = [add]
            cur = len(add)
        else:
            buf.append(add)
            cur += len(add)
    if buf:
        parts.append(" ".join(buf).strip())

    out = []
    for p in parts:
        out.append(translate_text_nllb_api(p, target_lang_code))
    return " ".join(out).strip()

from elevenlabs.client import ElevenLabs

def text_to_speech_elevenlabs(
    text: str,
    voice_id: str = "JBFqnCBsd6RMkjVDRZzb",
    model_id: str = "eleven_multilingual_v2",
    filename: str = "output.mp3",
) -> Optional[str]:
    """
    Stream TTS from ElevenLabs and save to MP3. Returns local filename or None on error.
    """
    try:
        api_key = os.getenv("ELEVENLABS_API_KEY")
        if not api_key or api_key.startswith("eleven_XXXX"):
            raise RuntimeError("Missing ELEVENLABS_API_KEY environment variable.")

        eleven = ElevenLabs(api_key=api_key)
        audio_stream = eleven.text_to_speech.stream(
            text=text,
            voice_id=voice_id,
            model_id=model_id,
        )
        with open(filename, "wb") as f:
            for chunk in audio_stream:
                if isinstance(chunk, bytes):
                    f.write(chunk)
        return filename
    except Exception as e:
        print(f"[ERROR] ElevenLabs TTS failed: {e}")
        return None


translate_runnable = RunnableLambda(
    lambda x: {
        "translated_text": translate_chunked_nllb(x["text"], x["lang"]),
        "lang": x["lang"],
        "require_audio": x.get("require_audio", True),
        "voice_id": x.get("voice_id", "JBFqnCBsd6RMkjVDRZzb"),
        "model_id": x.get("model_id", "eleven_multilingual_v2"),
    }
)

def maybe_tts(input_dict):
    if input_dict["require_audio"] and input_dict["translated_text"]:
        out_file = f"tts_{input_dict['lang']}.mp3"
        audio_path = text_to_speech_elevenlabs(
            input_dict["translated_text"],
            voice_id=input_dict["voice_id"],
            model_id=input_dict["model_id"],
            filename=out_file,
        )
    else:
        audio_path = None
    return {"translated_text": input_dict["translated_text"], "audio_path": audio_path}

tts_runnable = RunnableLambda(maybe_tts)
translation_pipeline = translate_runnable | tts_runnable





In [20]:
if __name__ == "__main__":
    payload = {
        "text": "Your crop is affected by leaf blight. Apply this fungicide.",
        "lang": "hi",
        "require_audio": True,

    }
    out = translation_pipeline.invoke(payload)
    print("Translated:", out["translated_text"])
    print("Audio file:", out["audio_path"])

    try:
        from IPython.display import Audio, display
        if out["audio_path"]:
            display(Audio(out["audio_path"]))
    except Exception:
        pass

Translated: आपकी फसल पत्तियों के रोग से प्रभावित है। यह कवकनाशक लगाएं।
Audio file: tts_hi.mp3
